In [1]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 573.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 26.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=5286328988f9a58aaa8ac5962e01d2781b844a9cd7bfd073648b24931308a3a0
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [2]:
import pandas as pd
import string
import re
import nltk
import pymorphy2
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import numpy as np

In [3]:
pd.set_option('display.max_colwidth', 100)

Методы предобработки

In [4]:
def remove_punctuation(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct

def tokenise(text):
    tokens = re.split('\W+', text)
    return tokens

def remove_stopwords(tokenised_list):
    stopwords = nltk.corpus.stopwords.words('russian')
    filtered_text = [word for word in tokenised_list if word not in stopwords]
    return filtered_text

def stemming(tokenised_text):
    ps = nltk.SnowballStemmer('russian')
    processed_text = [ps.stem(word) for word in tokenised_text]
    return processed_text

def lemmatizing(tokenized_text):
    ma = pymorphy2.MorphAnalyzer()
    processed_text = [ma.parse(word)[0].normal_form for word in tokenized_text]
    return processed_text

def clean_text(text):
    text = remove_punctuation(text)
    tokens = tokenise(text)
    processed_text = lemmatizing(tokens)
    return processed_text

Тестирование метода random forest

In [6]:
data = pd.read_csv("requests_gpt.csv", encoding='windows-1251', sep = ';')
data.columns = ['label', 'requests']

In [7]:
# TF-IDF для векторизации
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['requests'])

X_features = pd.DataFrame(X_tfidf.toarray())

X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size = 0.3)

In [8]:
# вывод характеристик для различных гиперпараметров
def train_RF(n_est, depth):
    rf = RandomForestClassifier(n_estimators = n_est, max_depth=depth, n_jobs=-1)
    rf_model = rf.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred)
    print('Est: {} / Depth: {} ----- Precision: {} / Recall: {} / Accuracy: {}'.format(
        n_est, depth, np.round(precision, 3), np.round(recall,3),
        round((y_pred==y_test).sum() / len(y_pred), 3)))

for n_est in [10, 50, 100]:
    for depth in [10, 20, 30, None]:
        train_RF(n_est, depth)

Est: 10 / Depth: 10 ----- Precision: [0.429 0.857 1.   ] / Recall: [1.   0.75 0.5 ] / Accuracy: 0.706
Est: 10 / Depth: 20 ----- Precision: [0.286 0.667 0.75 ] / Recall: [0.667 0.5   0.5  ] / Accuracy: 0.529
Est: 10 / Depth: 30 ----- Precision: [0.5   1.    0.833] / Recall: [1.    0.625 0.833] / Accuracy: 0.765
Est: 10 / Depth: None ----- Precision: [0.375 0.8   1.   ] / Recall: [1.    0.5   0.667] / Accuracy: 0.647
Est: 50 / Depth: 10 ----- Precision: [0.429 1.    1.   ] / Recall: [1.    0.625 0.833] / Accuracy: 0.765
Est: 50 / Depth: 20 ----- Precision: [0.6   0.875 1.   ] / Recall: [1.    0.875 0.667] / Accuracy: 0.824
Est: 50 / Depth: 30 ----- Precision: [0.5 1.  1. ] / Recall: [1.    0.75  0.833] / Accuracy: 0.824
Est: 50 / Depth: None ----- Precision: [0.429 0.833 1.   ] / Recall: [1.    0.625 0.667] / Accuracy: 0.706
Est: 100 / Depth: 10 ----- Precision: [0.5   0.857 1.   ] / Recall: [1.    0.75  0.667] / Accuracy: 0.765
Est: 100 / Depth: 20 ----- Precision: [0.5 1.  1. ] / Recal

In [9]:
# работа с лучшим вариантом
rf = RandomForestClassifier(n_estimators = 50, max_depth=20, n_jobs=-1)
rf_model = rf.fit(X_train, y_train)
print(rf_model.predict(X_test))

[1 1 2 1 0 2 2 1 0 1 2 0 1 2 0 0 0]
